### Loading Packages

In [33]:
library(dplyr)

current_date <- as.Date("2025-01-07")


In [34]:
cbbdata::cbd_login(username = "Naki", password = "Duke@123")
data <- cbbdata::cbd_torvik_team_factors(year = 2025, no_bias = TRUE) 
print(data)

API Key set!# A tibble: 364 x 31
   team   conf  games  wins losses adj_t adj_o adj_d barthag   efg def_efg   ftr
   <chr>  <chr> <dbl> <dbl>  <dbl> <dbl> <dbl> <dbl>   <dbl> <dbl>   <dbl> <dbl>
 1 Auburn SEC      14    13      1  69.4  135.  93.5   0.985  59.3    43.9  30.5
 2 Houst~ B12      13    10      3  63    121.  84.4   0.985  53.2    40    27.5
 3 Duke   ACC      14    12      2  66.7  121.  86.3   0.981  56      41.7  29.2
 4 Tenne~ SEC      14    14      0  66.4  119.  85.6   0.977  55.7    40.8  33  
 5 Iowa ~ B12      13    12      1  70.6  125.  94.4   0.960  56.7    47.3  40.2
 6 Gonza~ WCC      16    12      4  71.5  124.  94.3   0.960  55.4    46    30.1
 7 Illin~ B10      14    11      3  73.3  120.  91.5   0.957  54.5    42.8  37.3
 8 Alaba~ SEC      14    12      2  75    125.  96.0   0.955  55.2    46    41.1
 9 Flori~ SEC      14    13      1  70.8  125.  96.2   0.953  55.3    43.7  32.8
10 Kansas B12      13    10      3  70.4  116.  90.4   0.944  55      43.4  

## Extract ACC and B10 Teams from list

In [35]:
# # Filter teams in ACC and B10 and extract into separate lists
acc_teams <- data |>
  filter(conf == "ACC") |>
  pull(team) # Extract the 'team' column as a list

b10_teams <- data |>
  filter(conf == "B10") |>
  pull(team) # Extract the 'team' column as a list

SEC_teams <- data |>
  filter(conf == "SEC") |>
  pull(team) # Extract the 'team' column as a list

# espn_top25_week8 <- c(
#   "Tennessee", "Auburn", "Iowa St.", "Duke", "Alabama", "Florida", 
#   "Kansas", "Marquette", "Oregon", "Kentucky", "Connecticut", "Oklahoma", 
#   "Texas A&M", "Gonzaga", "Houston", "Mississippi", "Cincinnati", 
#   "Michigan St.", "Mississippi St.", "San Diego St.", 
#   "Purdue", "UCLA", "Arkansas", "Illinois", "Baylor"
# )

# extra_games <- c("Colorado St.", "Georgia St.", "North Carolina Central", "Iowa St.", 
#   "Richmond", "Boise St.", "Nevada", "Eastern Kentucky",  "Iona", "Texas St.", 
#   "Army", "Northeastern", "Northwestern"
#   )

unique_teams <- unique(c(acc_teams, b10_teams, SEC_teams))

# print(acc_teams)
# print(b10_teams)
# print(b12_teams)
# print(espn_top25_week8)
# print(unique_teams)

#unique_teams <- c("Duke", "North Carolina", "North Carolina St.")

## Extract Relevant Factors

In [36]:
selected_data_columns <- data |> 
  dplyr::select(Team = team, 
                ADJO = adj_o, 
                ADJD = adj_d,
                EFG_pct = efg, 
                TO_Pct = tov_rate, 
                OR_Pct = oreb_rate,
                FTR_Pct = ftr, 
                ThreePointer_Pct = three_pt_pct)

print(selected_data_columns)
print(which(is.na(selected_data_columns), arr.ind = TRUE))

# A tibble: 364 x 8
   Team       ADJO  ADJD EFG_pct TO_Pct OR_Pct FTR_Pct ThreePointer_Pct
   <chr>     <dbl> <dbl>   <dbl>  <dbl>  <dbl>   <dbl>            <dbl>
 1 Auburn     135.  93.5    59.3   13.5   35.6    30.5             37.7
 2 Houston    121.  84.4    53.2   15     37.3    27.5             41.4
 3 Duke       121.  86.3    56     15.1   34.6    29.2             35.9
 4 Tennessee  119.  85.6    55.7   16.4   40      33               35.7
 5 Iowa St.   125.  94.4    56.7   13.8   34.5    40.2             35.2
 6 Gonzaga    124.  94.3    55.4   13.7   33      30.1             34.6
 7 Illinois   120.  91.5    54.5   15.8   36.4    37.3             34  
 8 Alabama    125.  96.0    55.2   16.4   38.2    41.1             31.6
 9 Florida    125.  96.2    55.3   15.2   41.4    32.8             34.6
10 Kansas     116.  90.4    55     14.7   30.7    20.9             35.2
# i 354 more rows
     row col


## Cleaning Season Schedule

In [37]:
all_games <- cbbdata::cbd_torvik_season_schedule(year = 2025)
all_games <- all_games |>
  mutate(Location = ifelse(neutral, "N", "H")) |>
  select(-neutral, -type, -year, -game_id) |>
  rename(Team = home,
         Opponent = away,
         )

print(which(is.na(all_games), arr.ind = TRUE))
print(all_games)

     row col
            date              Team            Opponent Location
          <Date>            <char>              <char>   <char>
   1: 2024-11-04             Tulsa    Northeastern St.        H
   2: 2024-11-04         Tennessee        Gardner Webb        H
   3: 2024-11-04 Houston Christian               Avila        H
   4: 2024-11-04         Stonehill      Framingham St.        H
   5: 2024-11-04    North Carolina                Elon        H
  ---                                                          
4100: 2025-01-30        Lindenwood    SIU Edwardsville        H
4101: 2025-01-30          Lipscomb    Eastern Kentucky        H
4102: 2025-01-30      LIU Brooklyn Central Connecticut        H
4103: 2025-01-30    Long Beach St.           UC Irvine        H
4104: 2025-01-30          Marshall         Georgia St.        H


## Finding Games for Particluar Data

In [38]:
upcoming_games <- all_games |>
  filter(date >= current_date & date <= as.Date("2025-01-09"))
print(upcoming_games)

required_upcoming_games <- upcoming_games |>
  filter(Team %in% unique_teams | Opponent %in% unique_teams)
print(required_upcoming_games)

           date             Team          Opponent Location
         <Date>           <char>            <char>   <char>
  1: 2025-01-07 Eastern Illinois  SIU Edwardsville        H
  2: 2025-01-07          Florida         Tennessee        H
  3: 2025-01-07     San Jose St.          Utah St.        H
  4: 2025-01-07          Wyoming        New Mexico        H
  5: 2025-01-07           Baylor        Cincinnati        H
 ---                                                       
134: 2025-01-09      Utah Valley      Grand Canyon        H
135: 2025-01-09  Sam Houston St.           Liberty        H
136: 2025-01-09           Denver      North Dakota        H
137: 2025-01-09      Chattanooga    UNC Greensboro        H
138: 2025-01-09      Montana St. Northern Colorado        H
          date               Team        Opponent Location
        <Date>             <char>          <char>   <char>
 1: 2025-01-07            Florida       Tennessee        H
 2: 2025-01-07               Duke      Pitt

In [39]:
selected_data_columns$Team <- gsub("N\\.C\\. State", "North Carolina St.", selected_data_columns$Team)


# Extract stats for Team and Opponent
team_data <- required_upcoming_games |>
  left_join(selected_data_columns, by = c("Team" = "Team"))

opponent_data <- required_upcoming_games |>
  left_join(selected_data_columns, by = c("Opponent" = "Team")) |>
  rename_with(~ paste("Opp_", ., sep = ""), everything())

# Rename and remove unwanted columns
opponent_data <- opponent_data |>
  rename(Opponent = Opp_Opponent, Team = Opp_Team) |>
  select(-c("Opp_date", "Opp_Location"))

# Combine both datasets
final_data <- team_data |>
  left_join(opponent_data, by = c("Team", "Opponent"))

# Print the final result
print(final_data)

setdiff(required_upcoming_games$Team, selected_data_columns$Team)
setdiff(required_upcoming_games$Opponent, selected_data_columns$Team)


          date               Team        Opponent Location     ADJO      ADJD
        <Date>             <char>          <char>   <char>    <num>     <num>
 1: 2025-01-07            Florida       Tennessee        H 124.9994  96.23412
 2: 2025-01-07               Duke      Pittsburgh        H 121.3947  86.26055
 3: 2025-01-07           Miami FL     Florida St.        H 116.8911 113.40894
 4: 2025-01-07           Missouri             LSU        H 116.2121  99.96107
 5: 2025-01-07              Texas          Auburn        H 113.2094  97.54939
 6: 2025-01-07         Louisville         Clemson        H 115.0978  97.31895
 7: 2025-01-07               Iowa        Nebraska        H 119.2540 102.94572
 8: 2025-01-07            Georgia        Kentucky        H 112.6771  95.68103
 9: 2025-01-07         Vanderbilt Mississippi St.        H 117.6488  99.42909
10: 2025-01-07           Syracuse    Georgia Tech        H 108.7022 108.43643
11: 2025-01-07     North Carolina             SMU        H 116.6

character(0)

character(0)

In [40]:
file_to_write <- "test_file_Jan07to09.csv"
write.csv(final_data, file_to_write, row.names = FALSE)